In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import time
import csv

base_url = "https://www.technodom.kz"
start_path = "/catalog/smartfony-i-gadzhety/smartfony-i-telefony/smartfony"
max_pages = 50  # You can change this to a higher or lower number

all_products = []

for page_number in range(1, max_pages + 1):
    current_url = f"{base_url}{start_path}?page={page_number}"
    print(f"\n🔄 Parsing page {page_number}: {current_url}")

    response = requests.get(current_url)
    if response.status_code != 200:
        print(f"⚠️ Error {response.status_code} while requesting the page.")
        break

    soup = BeautifulSoup(response.text, "html.parser")

    products = soup.find_all('a', class_=re.compile(r'ProductItem_itemLink.*'))
    if not products:
        print("❌ No products found, stopping.")
        break

    for idx, product in enumerate(products, 1):
        product_title_tag = product.find('p', {'data-testid': 'product-title'})
        title = product_title_tag.text.strip() if product_title_tag else "Title not found"

        link = product.get('href')
        full_link = f"{base_url}{link}"

        print(f"\n📱 Product: {title}")
        print(f"🔗 Link: {full_link}")

        product_reviews = []

        try:
            product_page_response = requests.get(full_link)
            if product_page_response.status_code != 200:
                print(f"⚠️ Error {product_page_response.status_code} while loading product page.")
                continue

            product_soup = BeautifulSoup(product_page_response.text, "html.parser")
            reviews = product_soup.find_all('div', class_=re.compile(r'Review_block.*'))

            if reviews:
                print("💬 Reviews:")
                for review in reviews:
                    review_comment = review.find('span', class_='Review_textSummary__fz08A')
                    review_text = review_comment.text.strip() if review_comment else 'No review text.'
                    print(f" - Review: {review_text}")
                    product_reviews.append(review_text)
            else:
                print("❌ No reviews found.")

        except Exception as e:
            print(f"⚠️ Error while processing the product: {e}")

        all_products.append({
            "Title": title,
            "Link": full_link,
            "Reviews": " ||| ".join(product_reviews) if product_reviews else "No reviews"
        })

        print("-" * 60)
        time.sleep(1)

print(f"\n💾 Saving to CSV file...")

csv_filename = "technodom_smartphones.csv"
with open(csv_filename, mode='w', encoding='utf-8', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=["Title", "Link", "Reviews"])
    writer.writeheader()
    writer.writerows(all_products)

print(f"✅ Data saved to file: {csv_filename}")


🔄 Parsing page 1: https://www.technodom.kz/catalog/smartfony-i-gadzhety/smartfony-i-telefony/smartfony?page=1

📱 Product: Смартфон Apple iPhone 13 4/128GB Midnight
🔗 Link: https://www.technodom.kz/p/smartfon-gsm-apple-iphone-13-128gb-thx-61-12-5-midnight-252945?source=catalog&category_code=smartfony-i-gadzhety%2Fsmartfony-i-telefony%2Fsmartfony
💬 Reviews:
 - Review: отличный смартфон всем советуюу
 - Review: Очень хороший телефон 👍🏻👍🏻
 - Review: Супер все отлично
 - Review: Отлично работает, всё нравится
------------------------------------------------------------

📱 Product: Смартфон Huawei Nova Y72s 256GB Черный
🔗 Link: https://www.technodom.kz/p/smartfon-gsm-huawei-nova-y72s-8-256gb-cherniy-287914?source=catalog&category_code=smartfony-i-gadzhety%2Fsmartfony-i-telefony%2Fsmartfony
❌ No reviews found.
------------------------------------------------------------

📱 Product: Смартфон Samsung Galaxy A06 4/64GB Black
🔗 Link: https://www.technodom.kz/p/smartfon-gsm-samsung-galaxy-a06-464

In [11]:
!pip install deep-translator

In [13]:
import csv
import time
import pandas as pd
from deep_translator import GoogleTranslator

input_file = "technodom_smartphones.csv"
output_file = "technodom_reviews_for_nlp.csv"
final_file = "technodom_reviews_cleaned_final.csv"

# Step 1: Read and prepare reviews for translation
all_reviews = []
with open(input_file, mode='r', encoding='utf-8') as file:
    reader = csv.DictReader(file)
    for row in reader:
        reviews_raw = row["Reviews"]  # "Reviews" column in Russian
        if reviews_raw != "No reviews":  # Skip "No reviews"
            split_reviews = [r.strip() for r in reviews_raw.split("|||") if r.strip()]
            all_reviews.extend(split_reviews)

# Remove duplicates and empty strings
all_reviews = list(set([r for r in all_reviews if r]))

# Save reviews to a new CSV file with just one column: "review"
with open(output_file, mode='w', encoding='utf-8', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["review"])
    for review in all_reviews:
        writer.writerow([review])

print(f"✅ Reviews saved to NLP-ready file: {output_file}")

# Step 2: Translate reviews using Google Translate
translator = GoogleTranslator(source='auto', target='en')
translated_rows = []

with open(output_file, mode='r', encoding='utf-8') as file:
    reader = csv.DictReader(file)
    for row in reader:
        review = row['review']
        try:
            translated = translator.translate(review)
            translated_rows.append({"original": review, "translated": translated})
            print(f"✅ Translated: {translated}")
            time.sleep(0.5)  # to avoid hitting Google’s rate limit
        except Exception as e:
            print(f"⚠️ Translation error: {e}")
            translated_rows.append({"original": review, "translated": "TRANSLATION_ERROR"})

# Step 3: Clean the data
df = pd.DataFrame(translated_rows)

# Drop rows where the "translated" column has missing values or errors
df_cleaned = df[df["translated"].notna() & (df["translated"] != "TRANSLATION_ERROR")]

# Step 4: Modify the first line (replace column names) and remove duplicates
df_cleaned = df_cleaned.drop_duplicates()

# Step 5: Remove paragraphs (newlines) and replace them with ". "
df_cleaned['translated'] = df_cleaned['translated'].str.replace('\n', '. ', regex=False)

# Save the final cleaned data
df_cleaned.to_csv(final_file, index=False, quoting=1)

print(f"✅ Final cleaned data saved to {final_file}")

✅ Reviews saved to NLP-ready file: technodom_reviews_for_nlp.csv
✅ Translated: inexpensive
✅ Translated: works)
✅ Translated: The phone is excellent at an affordable price does not hang does not lag supports heavy games. I recommend
✅ Translated: Does not contain built -in applications from Google. A large amount of RAM and internal memory. Enlightenment camera.
✅ Translated: Very good phone
✅ Translated: a lot of attachments, diverse social networks are loaded
✅ Translated: powerful
✅ Translated: The phone is fire, I don’t know at the expense of games, I took my wife, for two days it calmly holds without aim
✅ Translated: A good flagship
✅ Translated: Everything is fine, works with a bang.
✅ Translated: per 100 💯
✅ Translated: While my daughter is happy
✅ Translated: Great smartphone
✅ Translated: Amir is the best seller
✅ Translated: Apple is Apple
✅ Translated: Great phone, very nimble, cool camera and super display. Face ID for Android!
✅ Translated: Cool phone, camera, charging, n

In [14]:
!aws s3 cp technodom_reviews_cleaned_final.csv s3://zanggar/technodom_reviews/technodom_reviews_cleaned_final

upload: ./technodom_reviews_cleaned_final.csv to s3://zanggar/technodom_reviews/technodom_reviews_cleaned_final
